In [59]:
using LinearAlgebra
using SparseArrays
include("bArnoldi.jl")
include("arnoldi.jl")

arnoldi (generic function with 1 method)

In [60]:
n = 10
M = 3
L = 2
#^^^^^^^^^^^^^^^^^^^^^^^#
#  A*X_σ - X_σ * D = B  #
#_______________________#
# LHS
#   A
λ = [i+10 for i in 1:n]
a = randn(n, n)
A = UpperTriangular(a) - diagm(diag(a)) + diagm(λ)
#   Shift
σ = rand(1:9, L)
D = diagm(σ) 
Γ = .7
# RHS
B = randn(n, L);

In [61]:
X_exact = zeros(n, L)
for l=1:L
    lhs = A # - D[l, l]*I
    b = B[:, l]
    X_exact[:, l] = lhs \ b
end

In [62]:
# Naive: Standard Arnoldi in a for-loop
X_σ_naive = zeros(n, L) 
for l=1:L
    lhs = A # - D[l, l]*I
    b = B[:, l]
    Q, H = arnoldi(lhs, b, M)
    for m in 1:M
        s = vcat(norm(b), zeros(m, 1))
        z = H[1:m+1, 1:m] \ s
        x = Q[:, 1:m] * z
        X_σ_naive[:, l] = x
    end
end
norm(X_exact - X_σ_naive)

0.0011250872790743098

In [63]:
# Block Arnoldi
X_σ = zeros(n, L) 
Res = zeros(n, L)
W, H = bArnoldi(A, B, M)
@show norm(A*W[:,1:M*L] - W*H)

norm(A * W[:, 1:M * L] - W * H) = 4.268339196329169e-15


4.268339196329169e-15

In [64]:
f = qr(B)
for m in 1:M

    print("\n")
    print(m)
    print("\n")

    #s = vcat(f.Q[:, 1:m*L]' * B, zeros(m*L, L))            # s = Q' B
    s = f.Q[:, 1:m*L]' * B
    z = H[1:L*m, 1:L*m] \ s      # z = H \ s
    x = W[:, 1:L*m] * z              # x = Q_m z

    @show size(H[1:L*m, 1:L*m]), size(z), size(s)
    @show size(x)
    # Populate results
    #Res[m, :] = norm(B - A*x)
    
    X_σ = x
end
norm(X_exact - X_σ)


1
(size(H[1:L * m, 1:L * m]), size(z), size(s)) = ((2, 2), (2, 2), (2, 2))
size(x) = (10, 2)

2
(size(H[1:L * m, 1:L * m]), size(z), size(s)) = ((4, 4), (4, 2), (4, 2))
size(x) = (10, 2)

3
(size(H[1:L * m, 1:L * m]), size(z), size(s)) = ((6, 6), (6, 2), (6, 2))
size(x) = (10, 2)


0.0008247865458325761